In [2]:
#Referencia a todas las  librerias usadas
import pandas as pd
import datetime
import sys
import numpy
import json
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [3]:
#Lectura de un GEOJSON

with open('original_data/milan_grid/milano-grid.geojson') as f:
    dataGeo = json.load(f)

type(dataGeo)

dict

In [4]:
#Obtencion de las estaciones de aire de Milan
dataAirLegend = pd.read_csv('original_data/milan_air/mi_air_legend.csv', index_col=False)
dataAirLegend.describe()

,IdSensore,Idstazione,Quota,Utm_Nord,UTM_Est,lat,lng
count,951.000000,951.000000,937.000000,9.510000e+02,951.000000,951.000000,951.000000
mean,8519.665615,642.188223,182.058698,5.040880e+06,546150.444795,45.518210,9.590074
std,3913.402809,168.545203,179.380027,3.282929e+04,49152.027765,0.297396,0.626402
min,5504.000000,501.000000,11.000000,4.975705e+06,477613.000000,44.924694,8.712691
25%,5818.500000,555.000000,85.000000,5.017060e+06,511522.000000,45.306013,9.146666
50%,6764.000000,600.000000,147.000000,5.041100e+06,537706.000000,45.520009,9.484261
75%,10151.000000,670.000000,222.000000,5.059472e+06,578919.000000,45.688657,10.006202
max,30166.000000,1374.000000,1225.000000,5.147136e+06,686934.000000,46.469524,11.372345


In [5]:
dataAirLegend

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng,location
0,10179,Biossido di Azoto,µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,29/11/2006,01/01/2017,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
1,10177,Ossidi di Azoto,µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,29/11/2006,01/01/2017,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
2,10185,PM10 (SM2005),µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,01/01/2007,01/01/2017,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
3,10399,Particelle sospese PM2.5,µg/m³,583,Bergamo - via Meucci,249.0,BG,Bergamo,N,20/12/2008,NaN,5059922,550116,45.691037,9.643651,"(45.69103740547214, 9.643650579461385)"
4,6642,Particolato Totale Sospeso,µg/m³,542,Meda,222.0,MB,Meda,S,01/11/1995,04/02/1998,5056480,512203,45.661762,9.156644,"(45.66176219189168, 9.156643984551597)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,6312,Ossidi di Azoto,µg/m³,502,Milano - via Juvara,122.0,MI,Milano,S,06/05/1977,11/06/2007,5035546,517377,45.473226,9.222315,"(45.473225708577175, 9.222315138737468)"
947,5572,Biossido di Azoto,µg/m³,569,Sondrio - via Mazzini,307.0,SO,Sondrio,N,01/11/1993,NaN,5113078,567173,46.167967,9.870144,"(46.16796681227828, 9.87014407497457)"
948,10355,Monossido di Carbonio,mg/m³,516,Robecchetto,135.0,MI,Robecchetto con Induno,S,28/02/2007,01/01/2017,5040503,481768,45.517822,8.766562,"(45.51782219008838, 8.766562109244964)"
949,12577,Particelle sospese PM2.5,µg/m³,627,Cremona - p.zza Cadorna,39.0,CR,Cremona,N,01/01/2012,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"


In [6]:
dataAirLegend['DataStop'] = pd.to_datetime(dataAirLegend['DataStop'], format='%d/%m/%Y', errors='ignore')

In [7]:
dataAirLegend = dataAirLegend[(dataAirLegend['DataStop'].isnull()) | (dataAirLegend['DataStop'] > '2013-11-01')] 

In [8]:
dataAirLegend

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng,location
0,10179,Biossido di Azoto,µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,29/11/2006,2017-01-01,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
1,10177,Ossidi di Azoto,µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,29/11/2006,2017-01-01,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
2,10185,PM10 (SM2005),µg/m³,704,Sermide - via Dalla Chiesa,11.0,MN,Sermide e Felonica,S,01/01/2007,2017-01-01,4986023,680789,45.004614,11.294015,"(45.004613731114105, 11.294014707706845)"
3,10399,Particelle sospese PM2.5,µg/m³,583,Bergamo - via Meucci,249.0,BG,Bergamo,N,20/12/2008,NaT,5059922,550116,45.691037,9.643651,"(45.69103740547214, 9.643650579461385)"
5,9859,Monossido di Carbonio,mg/m³,673,Voghera - via Pozzoni,90.0,PV,Voghera,N,05/07/2005,NaT,4982900,500665,44.999546,9.008437,"(44.99954598914026, 9.008437381815693)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,5631,Biossido di Zolfo,µg/m³,546,Magenta,137.0,MI,Magenta,N,22/04/1995,NaT,5034328,490635,45.462416,8.880210,"(45.462415791106615, 8.880210433125571)"
947,5572,Biossido di Azoto,µg/m³,569,Sondrio - via Mazzini,307.0,SO,Sondrio,N,01/11/1993,NaT,5113078,567173,46.167967,9.870144,"(46.16796681227828, 9.87014407497457)"
948,10355,Monossido di Carbonio,mg/m³,516,Robecchetto,135.0,MI,Robecchetto con Induno,S,28/02/2007,2017-01-01,5040503,481768,45.517822,8.766562,"(45.51782219008838, 8.766562109244964)"
949,12577,Particelle sospese PM2.5,µg/m³,627,Cremona - p.zza Cadorna,39.0,CR,Cremona,N,01/01/2012,NaT,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"


In [15]:
#Obtención de datos de sensores de aire y relación con cellId
sliceDataAir = pd.DataFrame({})
i = 0
j = 1
for dataAirSensor in dataAirLegend.itertuples(index=False):
    #print (dataAirSensor.lat)
    coord = dataAirSensor.location.split(",")
    lat = coord[0].replace("(", "")
    long = coord[1].replace(")", "")
    point = Point([float(long), float(lat)])
    print (point)
    for featureData in dataGeo['features']:
        datos = featureData['geometry']['coordinates']
        polygon =  Polygon(datos[0])
        cellId = featureData['properties']['cellId']
        if (polygon.contains (point)):
            print (polygon)
            print (cellId)
            sliceSum2 = pd.DataFrame(np.array([[dataAirSensor.IdSensore, dataAirSensor.NomeStazione, dataAirSensor.lng, dataAirSensor.lat, dataAirSensor.NomeTipoSensore, dataAirSensor.UnitaMisura, int(cellId)]]), 
                                               columns=['sensorID', 'streetName', 'sensorCoordX', 'sensorCoordY', 'sensorType', 'unitMeasure', 'cellId'])
            sliceDataAir = sliceDataAir.append(sliceSum2)
            continue
            #print (featureTweet['properties']['cellId'])

POINT (11.29401470770684 45.00461373111411)
POINT (11.29401470770684 45.00461373111411)
POINT (11.29401470770684 45.00461373111411)
POINT (9.643650579461385 45.69103740547214)
POINT (9.008437381815693 44.99954598914026)
POINT (10.79556411370788 45.16056833010576)
POINT (8.736496578379215 45.52655086994247)
POINT (9.04006309067821 45.62621896083601)
POINT (9.395577066893223 45.85019119457854)
POINT (9.601223338785262 45.64970251236247)
POINT (9.395577066893223 45.85019119457854)
POINT (9.265020964805544 45.75111126161782)
POINT (10.74263625117323 45.20817903938493)
POINT (10.23178590747722 45.53952943495599)
POINT (9.497538167653424 45.91278752030292)
POINT (9.273573092713274 45.58026348442243)
POINT (11.13703400729984 45.07331115573196)
POINT (8.736496578379215 45.52655086994247)
POINT (9.164638300630086 45.19467609907785)
POINT (9.592009994888425 45.51933498138716)
POINT (10.37983166852242 45.12979322367744)
POINT (10.78140770009565 45.15174337957659)
POINT (9.69814824935254 45.159372

In [16]:
sliceDataAir

,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,cellId
0,20005,Milano - via Senato,9.197460360112533,45.470499014097,BlackCarbon,µg/m³,5362
0,6328,Milano - viale Marche,9.190933555313624,45.496316443651025,Ossidi di Azoto,µg/m³,6660
0,10282,Milano - Pascal Città Studi,9.235491038497502,45.47899606168744,Ozono,µg/m³,5775
0,5542,Milano - viale Liguria,9.167944501742676,45.44385765356493,Biossido di Azoto,µg/m³,4153
0,5518,Pero,9.089972020337333,45.50985226679251,Biossido di Azoto,µg/m³,7227
...,...,...,...,...,...,...,...
0,5507,Sesto S.Giovanni,9.236107923411636,45.53476576509224,Biossido di Azoto,µg/m³,8475
0,6300,Cormano,9.162613653151322,45.55104194437242,Ossidi di Azoto,µg/m³,9151
0,5531,Milano - Verziere,9.195324807668856,45.46334674066655,Biossido di Azoto,µg/m³,5062
0,5539,Arese,9.080725831284644,45.543433254501636,Biossido di Azoto,µg/m³,8823


In [17]:
CellId3x3 = pd.read_csv('transformed_data/CellId3x3.csv')
sliceDataAir['cellId'] = sliceDataAir['cellId'].astype(int)
SensorIDCellId3x3 = sliceDataAir[sliceDataAir['cellId'].isin(CellId3x3['# CellId'].values)]['sensorID'].values

In [18]:
SensorIDCellId3x3

array(['6328', '5506', '17127', '5827', '5504', '5841', '6372', '6062'],
      dtype=object)

In [19]:
sensorID3x3 = sliceDataAir[sliceDataAir['sensorID'].isin(SensorIDCellId3x3)]
sensorID3x3

,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,cellId
0,6328,Milano - viale Marche,9.190933555313624,45.496316443651025,Ossidi di Azoto,µg/m³,6660
0,5506,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Biossido di Azoto,µg/m³,5644
0,17127,Milano - viale Marche,9.190933555313624,45.496316443651025,Benzene,µg/m³,6660
0,5827,Milano - viale Marche,9.190933555313624,45.496316443651025,Monossido di Carbonio,mg/m³,6660
0,5504,Milano - viale Marche,9.190933555313624,45.496316443651025,Biossido di Azoto,µg/m³,6660
0,5841,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Monossido di Carbonio,mg/m³,5644
0,6372,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Ossidi di Azoto,µg/m³,5644
0,6062,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Benzene,µg/m³,5644


In [57]:
sensorID3x3.loc[sensorID3x3['sensorType'] == 'Ossidi di Azoto', 'sensorType'] = 'NO' #'nitrogeOxides'
sensorID3x3.loc[sensorID3x3['sensorType'] == 'Biossido di Azoto', 'sensorType'] = 'NO2' #'nitrogenDioxide'
sensorID3x3.loc[sensorID3x3['sensorType'] == 'Monossido di Carbonio', 'sensorType'] = 'CO' #'carbonMonoxide'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [58]:
sensorID3x3

,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,cellId
0,6328,Milano - viale Marche,9.190933555313624,45.496316443651025,NO,µg/m³,6660
0,5506,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO2,µg/m³,5644
0,17127,Milano - viale Marche,9.190933555313624,45.496316443651025,Benzene,µg/m³,6660
0,5827,Milano - viale Marche,9.190933555313624,45.496316443651025,CO,mg/m³,6660
0,5504,Milano - viale Marche,9.190933555313624,45.496316443651025,NO2,µg/m³,6660
0,5841,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,CO,mg/m³,5644
0,6372,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO,µg/m³,5644
0,6062,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Benzene,µg/m³,5644


In [59]:
dataSensorAir = pd.read_csv('original_data/milan_air/air_2013.csv', index_col=False)
dataSensorAir.head()

,IdSensore,Data,Valore,Stato,idOperatore
0,10001,01/01/2013 00:00:00,341.8,VA,1
1,10002,01/01/2013 00:00:00,2.2,VA,1
2,10004,01/01/2013 00:00:00,148.5,VA,1
3,10005,01/01/2013 00:00:00,20.1,VA,1
4,10006,01/01/2013 00:00:00,1.9,VA,1


In [60]:
SensorIDCellId3x3

array(['6328', '5506', '17127', '5827', '5504', '5841', '6372', '6062'],
      dtype=object)

In [61]:
dataSensorAir = dataSensorAir[dataSensorAir['IdSensore'].isin(SensorIDCellId3x3)]
dataSensorAir

,IdSensore,Data,Valore,Stato,idOperatore
210,17127,01/01/2013 00:00:00,4.0,VA,1
221,5504,01/01/2013 00:00:00,85.7,VA,1
223,5506,01/01/2013 00:00:00,72.6,VA,1
436,5827,01/01/2013 00:00:00,3.1,VA,1
445,5841,01/01/2013 00:00:00,1.9,VA,1
...,...,...,...,...,...
4913290,5506,31/12/2013 23:00:00,77.5,VA,1
4913458,5827,31/12/2013 23:00:00,1.4,VA,1
4913465,5841,31/12/2013 23:00:00,2.2,VA,1
4913531,6328,31/12/2013 23:00:00,183.4,VA,1


In [62]:
#dataAirLegend[dataAirLegend['IdSensore'].isin(sensorIdAir)]

NameError: name 'sensorIdAir' is not defined

In [ ]:
#dataSensorAir

In [68]:
dataSensorAir['Data'] = pd.to_datetime(dataSensorAir['Data'])
dataSensorAir = dataSensorAir[(dataSensorAir['Data'] >= '2013-10-31 23:00:00') & (dataSensorAir['Data'] < '2014-01-01 00:00:00')]
dataSensorAir

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,IdSensore,Data,Valore,Stato,idOperatore
133677,17127,2013-11-01 00:00:00,3.4,VA,1
133688,5504,2013-11-01 00:00:00,75.3,VA,1
133690,5506,2013-11-01 00:00:00,63.9,VA,1
133904,5827,2013-11-01 00:00:00,2.9,VA,1
133914,5841,2013-11-01 00:00:00,1.4,VA,1
...,...,...,...,...,...
4913290,5506,2013-12-31 23:00:00,77.5,VA,1
4913458,5827,2013-12-31 23:00:00,1.4,VA,1
4913465,5841,2013-12-31 23:00:00,2.2,VA,1
4913531,6328,2013-12-31 23:00:00,183.4,VA,1


In [65]:
sensorID3x3

,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,cellId
0,6328,Milano - viale Marche,9.190933555313624,45.496316443651025,NO,µg/m³,6660
0,5506,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO2,µg/m³,5644
0,17127,Milano - viale Marche,9.190933555313624,45.496316443651025,Benzene,µg/m³,6660
0,5827,Milano - viale Marche,9.190933555313624,45.496316443651025,CO,mg/m³,6660
0,5504,Milano - viale Marche,9.190933555313624,45.496316443651025,NO2,µg/m³,6660
0,5841,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,CO,mg/m³,5644
0,6372,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO,µg/m³,5644
0,6062,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,Benzene,µg/m³,5644


In [76]:
sensorID3x3['sensorID'] = sensorID3x3['sensorID'].astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
#Juntamos todos los archivos generados anteriormente en uno solo
sliceSensorAir = pd.DataFrame({})
for dataAirValue in dataSensorAir.itertuples(index=False):
    print(dataAirValue[0])
    print(type(dataAirValue[0]))
    for dataAirSensor in sensorID3x3.itertuples(index=False):
        print(dataAirSensor[0])
        print(type(dataAirSensor[0]))
        if (dataAirValue[0] == dataAirSensor[0]):
            print("Entra")
            sliceSum2 = pd.DataFrame(np.array([[dataAirSensor.cellId, dataAirSensor.sensorID, dataAirSensor.streetName, dataAirSensor.sensorCoordX , dataAirSensor.sensorCoordY, dataAirSensor.sensorType , dataAirSensor.unitMeasure, dataAirValue.Data, dataAirValue.Valore]]), 
                                               columns=['cellId', 'sensorID', 'streetName', 'sensorCoordX', 'sensorCoordY', 'sensorType', 'unitMeasure', 'timeInstant', 'value' ])
            sliceSensorAir = sliceSensorAir.append(sliceSum2)

17127
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
17127
<class 'int'>
Entra
5827
<class 'int'>
5504
<class 'int'>
5841
<class 'int'>
6372
<class 'int'>
6062
<class 'int'>
5504
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
17127
<class 'int'>
5827
<class 'int'>
5504
<class 'int'>
Entra
5841
<class 'int'>
6372
<class 'int'>
6062
<class 'int'>
5506
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
Entra
17127
<class 'int'>
5827
<class 'int'>
5504
<class 'int'>
5841
<class 'int'>
6372
<class 'int'>
6062
<class 'int'>
5827
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
17127
<class 'int'>
5827
<class 'int'>
Entra
5504
<class 'int'>
5841
<class 'int'>
6372
<class 'int'>
6062
<class 'int'>
5841
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
17127
<class 'int'>
5827
<class 'int'>
5504
<class 'int'>
5841
<class 'int'>
Entra
6372
<class 'int'>
6062
<class 'int'>
6062
<class 'int'>
6328
<class 'int'>
5506
<class 'int'>
17127
<class 'int'>
5827
<class 'int'>
5504
<class '

In [80]:
sliceSensorAir

,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
0,6660,17127,Milano - viale Marche,9.190933555313624,45.496316443651025,Benzene,µg/m³,2013-11-01 00:00:00,3.4
0,6660,5504,Milano - viale Marche,9.190933555313624,45.496316443651025,NO2,µg/m³,2013-11-01 00:00:00,75.3
0,5644,5506,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO2,µg/m³,2013-11-01 00:00:00,63.9
0,6660,5827,Milano - viale Marche,9.190933555313624,45.496316443651025,CO,mg/m³,2013-11-01 00:00:00,2.9
0,5644,5841,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,CO,mg/m³,2013-11-01 00:00:00,1.4
...,...,...,...,...,...,...,...,...,...
0,5644,5506,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,NO2,µg/m³,2013-12-31 23:00:00,77.5
0,6660,5827,Milano - viale Marche,9.190933555313624,45.496316443651025,CO,mg/m³,2013-12-31 23:00:00,1.4
0,5644,5841,Milano - P.zza Zavattari,9.141786267186658,45.476063405097904,CO,mg/m³,2013-12-31 23:00:00,2.2
0,6660,6328,Milano - viale Marche,9.190933555313624,45.496316443651025,NO,µg/m³,2013-12-31 23:00:00,183.4


In [81]:
sliceSensorAir.to_csv('transformed_data/mi_sensorID_air.csv', encoding='utf-8')

In [82]:
trialDataAir = pd.read_csv('transformed_data/mi_sensorID_air.csv', index_col=False)
trialDataAir.head()

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
0,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-11-01 00:00:00,3.4
1,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 00:00:00,75.3
2,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 00:00:00,63.9
3,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 00:00:00,2.9
4,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 00:00:00,1.4


In [83]:
air17127 = trialDataAir[trialDataAir['sensorID'] == 17127]
air17127

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
0,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-11-01 00:00:00,3.4
146,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-01 00:00:00,2.3
292,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-11-02 00:00:00,1.6
436,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-02 00:00:00,2.0
580,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-11-03 00:00:00,2.4
...,...,...,...,...,...,...,...,...,...,...
8005,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-27 00:00:00,2.7
8151,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-28 00:00:00,3.8
8297,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-29 00:00:00,1.8
8443,0,6660,17127,Milano - viale Marche,9.190934,45.496316,Benzene,µg/m³,2013-12-30 00:00:00,3.2


In [84]:
air5504 = trialDataAir[trialDataAir['sensorID'] == 5504]
air5504

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
1,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 00:00:00,75.3
8,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 01:00:00,72.7
14,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 02:00:00,68.7
20,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 03:00:00,56.1
26,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-11-01 04:00:00,50.5
...,...,...,...,...,...,...,...,...,...,...
8705,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-12-31 19:00:00,62.8
8711,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-12-31 20:00:00,59.5
8717,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-12-31 21:00:00,57.8
8723,0,6660,5504,Milano - viale Marche,9.190934,45.496316,NO2,µg/m³,2013-12-31 22:00:00,59.9


In [85]:
air5506 = trialDataAir[trialDataAir['sensorID'] == 5506]
air5506

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
2,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 00:00:00,63.9
9,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 01:00:00,64.0
15,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 02:00:00,64.0
21,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 03:00:00,51.5
27,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-11-01 04:00:00,48.5
...,...,...,...,...,...,...,...,...,...,...
8706,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-12-31 19:00:00,92.3
8712,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-12-31 20:00:00,91.3
8718,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-12-31 21:00:00,91.7
8724,0,5644,5506,Milano - P.zza Zavattari,9.141786,45.476063,NO2,µg/m³,2013-12-31 22:00:00,86.9


In [86]:
air5827 = trialDataAir[trialDataAir['sensorID'] == 5827]
air5827

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
3,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 00:00:00,2.9
10,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 01:00:00,2.9
16,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 02:00:00,2.9
22,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 03:00:00,2.8
28,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-11-01 04:00:00,3.0
...,...,...,...,...,...,...,...,...,...,...
8707,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-12-31 19:00:00,1.4
8713,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-12-31 20:00:00,1.3
8719,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-12-31 21:00:00,1.3
8725,0,6660,5827,Milano - viale Marche,9.190934,45.496316,CO,mg/m³,2013-12-31 22:00:00,1.3


In [87]:
air6328 = trialDataAir[trialDataAir['sensorID'] == 6328]
air6328

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
6,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-11-01 00:00:00,211.4
12,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-11-01 01:00:00,212.4
18,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-11-01 02:00:00,205.3
24,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-11-01 03:00:00,162.1
30,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-11-01 04:00:00,171.2
...,...,...,...,...,...,...,...,...,...,...
8709,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-12-31 19:00:00,215.6
8715,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-12-31 20:00:00,185.3
8721,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-12-31 21:00:00,179.5
8727,0,6660,6328,Milano - viale Marche,9.190934,45.496316,NO,µg/m³,2013-12-31 22:00:00,205.1


In [88]:
air5841 = trialDataAir[trialDataAir['sensorID'] == 5841]
air5841

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
4,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 00:00:00,1.4
11,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 01:00:00,1.4
17,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 02:00:00,1.6
23,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 03:00:00,1.7
29,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-11-01 04:00:00,1.6
...,...,...,...,...,...,...,...,...,...,...
8708,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-12-31 19:00:00,2.1
8714,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-12-31 20:00:00,2.3
8720,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-12-31 21:00:00,2.2
8726,0,5644,5841,Milano - P.zza Zavattari,9.141786,45.476063,CO,mg/m³,2013-12-31 22:00:00,2.2


In [89]:
air6372 = trialDataAir[trialDataAir['sensorID'] == 6372]
air6372

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
7,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-11-01 00:00:00,204.7
13,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-11-01 01:00:00,231.2
19,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-11-01 02:00:00,262.8
25,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-11-01 03:00:00,209.7
31,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-11-01 04:00:00,170.6
...,...,...,...,...,...,...,...,...,...,...
8710,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-12-31 19:00:00,273.7
8716,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-12-31 20:00:00,310.4
8722,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-12-31 21:00:00,311.5
8728,0,5644,6372,Milano - P.zza Zavattari,9.141786,45.476063,NO,µg/m³,2013-12-31 22:00:00,311.8


In [90]:
air6062 = trialDataAir[trialDataAir['sensorID'] == 6062]
air6062

,Unnamed: 0,cellId,sensorID,streetName,sensorCoordX,sensorCoordY,sensorType,unitMeasure,timeInstant,value
5,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-11-01 00:00:00,1.8
151,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-01 00:00:00,0.8
297,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-11-02 00:00:00,0.6
440,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-02 00:00:00,0.6
585,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-11-03 00:00:00,1.2
...,...,...,...,...,...,...,...,...,...,...
8010,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-27 00:00:00,1.9
8156,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-28 00:00:00,2.8
8302,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-29 00:00:00,1.2
8448,0,5644,6062,Milano - P.zza Zavattari,9.141786,45.476063,Benzene,µg/m³,2013-12-30 00:00:00,2.1
